In [22]:
# -*- coding: utf-8 -*-

In [1]:
import pandas as pd
import numpy as np
import codecs
import json

In [2]:
brand_ldr_01 = pd.read_excel("./dataset/brand_ldr_01.xlsx")

In [3]:
brand_ldr_02 = pd.read_excel("./dataset/brand_ldr_02.xlsx")

In [4]:
unbrand_ldr = pd.read_excel("./dataset/unbrand_ldr.xlsx")

In [19]:
store = pd.read_excel("./dataset/store.xlsx")

In [4]:
brand_ldr = [brand_ldr_01,brand_ldr_02]
brand_ldr = pd.concat(brand_ldr)

In [40]:
path_count = brand_ldr['品牌'].groupby(brand_ldr['品牌']).count()
path_count = pd.DataFrame(path_count,columns=['品牌'])
path_count.to_json("./resultData/path_count.json",force_ascii=True)

In [26]:
path_count = brand_ldr['品牌'].groupby(brand_ldr['品牌']).count()
path_count = pd.DataFrame(path_count,columns=['品牌'])
path_count.to_excel("./resultData/path_count.xlsx")

In [36]:
course = brand_ldr.groupby([brand_ldr['出发城市'],brand_ldr['到达城市']]).count()
course.drop(['品牌','出发区域','到达区域','时效','线路类型','重货价格','重货折扣','轻货价格','轻货折扣','最第一票','最近一月成交量','评价数量'],inplace=True,axis=1)
course = course.reset_index()
course.columns = ["depart","arrive"]

In [37]:
stripstr = lambda x: x.strip() if isinstance(x, str) else x
course = course.applymap(stripstr)

In [38]:
cities = set()
for idx,row in course.groupby("depart").count().reset_index().iterrows():
    cities.add(row["depart"])
for idx,row in course.groupby("arrive").count().reset_index().iterrows():
    cities.add(row["arrive"])

In [3]:
import requests
#存储地理位置 结果为这样
#   {'北京市': {'lat': 39.92998577808024, 'lng': 116.39564503787867},
#     '重庆市': {'lat': 29.54460610888615, 'lng': 106.53063501341296}}

cityGeoCoord ={} 
# cities是城市名称的list eg:["北京市","重庆市"]

for city in cities:
    cityGeoCoord.setdefault(city,[])
    url = "http://api.map.baidu.com/geocoder/v2/?output=json&ak=自己的AK值&address="+city
    response =  requests.get(url)
    if response.status_code  is 200:
        if "result" in response.json():
            cityGeoCoord[city] =response.json()["result"]["location"]
            print(city,"地理位置获取成功")
        else:
            print(response.json())
    else:
        print(response.status_code )
# 要写文件也可以，最好写文件，第二次读取文件，以免重复请求消耗api次数
# with open("./cityGeoCoord.json",'wb') as fp:
#     fp.write(str(cityGeoCoord).encode("utf-8"))

北京市 地理位置获取成功
重庆市 地理位置获取成功


In [65]:
result_data ={
    "citiesFields": [
        "name",
        "city",
        "country",
        "longitude",
        "latitude"
    ],
    "courseFields": [
        "name",
        "city"
    ],
    "cities":[],
    "course":[],
    "routes":[]
}
for key in cityGeoCoord:
    if cityGeoCoord[key]:
        tempCity = []
#         print(cityGeoCoord[key])
        tempCity.append(key)
        tempCity.append(key)
        tempCity.append("中国")
        tempCity.append(cityGeoCoord[key]["lng"])
        tempCity.append(cityGeoCoord[key]["lat"])
        result_data["cities"].append(tempCity)

In [66]:
result_data

{'cities': [['高雄市', '高雄市', '中国', 80.45048607500866, 32.03623661318253],
  ['鹤岗市', '鹤岗市', '中国', 130.29247205063453, 47.33866590372683],
  ['聊城市', '聊城市', '中国', 115.98686913929107, 36.455828514727976],
  ['泰安市', '泰安市', '中国', 117.08941491713698, 36.18807775894815],
  ['滨州市', '滨州市', '中国', 117.96829241452845, 37.405313941825895],
  ['南沙群岛', '南沙群岛', '中国', 112.66030170907291, 4.974366192136791],
  ['甘孜藏族自治州', '甘孜藏族自治州', '中国', 101.96923206306379, 30.055144114355528],
  ['鹤壁市', '鹤壁市', '中国', 114.29776983802476, 35.75542587422399],
  ['怒江傈僳族自治州', '怒江傈僳族自治州', '中国', 98.85993204248207, 25.860676978165355],
  ['台南市', '台南市', '中国', 121.36052796411255, 38.96584875626212],
  ['泉州市', '泉州市', '中国', 118.60036234322938, 24.901652383991102],
  ['南阳市', '南阳市', '中国', 112.5428419005123, 33.0114195691165],
  ['清远市', '清远市', '中国', 113.04077334890835, 23.698468550422163],
  ['萍乡市', '萍乡市', '中国', 113.85991703300718, 27.639544222952306],
  ['鞍山市', '鞍山市', '中国', 123.0077633288837, 41.118743682153465],
  ['太原市', '太原市', '中国',

In [69]:
for idx,row in course[:40000].iterrows():
    if cityGeoCoord[row["depart"]] and cityGeoCoord[row["arrive"]] :
        if cityGeoCoord[row["depart"]] != cityGeoCoord[row["arrive"]]:
            tempCourse = []
            tempRoute = []

            tempCourse.append(row["depart"])
            tempCourse.append(row["arrive"])

            tempRoute.append([cityGeoCoord[row["depart"]]["lng"],cityGeoCoord[row["depart"]]["lat"]])
            tempRoute.append([cityGeoCoord[row["arrive"]]["lng"],cityGeoCoord[row["arrive"]]["lat"]])
            result_data["course"].append(tempCourse)
            result_data["routes"].append(tempRoute)
with open("./info.json",'wb') as fp:
    fp.write(json.dumps(result_data).encode("utf-8"))

In [30]:
course.columns = ["depart","arrive"]
course.to_json("./resultData/course.json",orient='index')

In [20]:
store = store.groupby(store['物流品牌'])

In [21]:
store = store.count().sort_values("城市编码")

In [23]:
store.drop(["城市名称","城市名称","网点名称","手机号码","电话号码","网点地址"],inplace=True,axis=1)

In [27]:
store.to_excel("./resultData/store_count.xlsx")

In [48]:
pathAndstore = pd.read_excel("./resultData/pathAndstore.xlsx")
pathAndstore = pathAndstore.set_index("品牌")

In [50]:
pathAndstore.to_json("./resultData/pathAndstore.json")

In [30]:
pathAndstore.to_json("./resultData/pathAndstore.json",orient='index')

In [5]:
brand_ldr_type =brand_ldr.groupby(["品牌","出发城市"]).count()

In [6]:
brand_ldr_type = brand_ldr.drop(['到达城市','出发区域','到达区域','时效','线路类型','重货价格','重货折扣','轻货价格','轻货折扣','最第一票','最近一月成交量','评价数量'],axis=1)

In [8]:
brand_ldr_type.to_excel("./resultData/brand_ldr_type.xlsx")

In [9]:
brand_ldr_type = pd.read_excel("./resultData/brand_ldr_type.xlsx")

In [11]:
brand_ldr_type = brand_ldr_type.groupby(["品牌","出发城市"]).count()

Empty DataFrame
Columns: []
Index: [(COSCO-AIR, 北京市), (COSCO-AIR, 西安市), (中通快运, 三明市), (中通快运, 三门峡市), (中通快运, 上海市), (中通快运, 上饶市), (中通快运, 中卫市), (中通快运, 临沧市), (中通快运, 丹东市), (中通快运, 丽水市), (中通快运, 丽江市), (中通快运, 乌兰察布市), (中通快运, 乐山市), (中通快运, 九江市), (中通快运, 保山市), (中通快运, 信阳市), (中通快运, 内江市), (中通快运, 包头市), (中通快运, 北京市), (中通快运, 南京市), (中通快运, 南充市), (中通快运, 南昌市), (中通快运, 南通市), (中通快运, 南阳市), (中通快运, 厦门市), (中通快运, 台州市), (中通快运, 吉安市), (中通快运, 吉林市), (中通快运, 吴忠市), (中通快运, 周口市), (中通快运, 呼伦贝尔市), (中通快运, 呼和浩特市), (中通快运, 咸阳市), (中通快运, 商丘市), (中通快运, 商洛市), (中通快运, 嘉兴市), (中通快运, 四平市), (中通快运, 固原市), (中通快运, 大理白族自治州), (中通快运, 大连市), (中通快运, 娄底市), (中通快运, 宁德市), (中通快运, 宁波市), (中通快运, 安康市), (中通快运, 安阳市), (中通快运, 安顺市), (中通快运, 宜宾市), (中通快运, 宝鸡市), (中通快运, 宿迁市), (中通快运, 岳阳市), (中通快运, 常州市), (中通快运, 常德市), (中通快运, 平顶山市), (中通快运, 广元市), (中通快运, 广安市), (中通快运, 广州市), (中通快运, 延安市), (中通快运, 延边朝鲜族自治州), (中通快运, 开封市), (中通快运, 张家界市), (中通快运, 徐州市), (中通快运, 德宏傣族景颇族自治州), (中通快运, 德阳市), (中通快运, 怒江傈僳族自治州), (中通快运, 成都市), (中通快运, 扬州市), (中通快运, 抚州市), (中通快运, 抚顺市), (中通快运, 文山壮族苗族自治州), (中通快运, 新乡市), (中通快运, 昆明市), (中通快运, 昭通市), (中通快运, 普洱市), (中通快运, 景德镇市), (中通快运, 曲靖市), (中通快运, 朝阳市), (中通快运, 本溪市), (中通快运, 杭州市), (中通快运, 株洲市), (中通快运, 楚雄彝族自治州), (中通快运, 榆林市), (中通快运, 永州市), (中通快运, 汉中市), (中通快运, 沈阳市), (中通快运, 泉州市), (中通快运, 泰州市), (中通快运, 泸州市), (中通快运, 洛阳市), (中通快运, 济源市), (中通快运, 淮安市), (中通快运, 温州市), (中通快运, 渭南市), (中通快运, 湖州市), (中通快运, 湘潭市), (中通快运, 漯河市), (中通快运, 漳州市), (中通快运, 濮阳市), (中通快运, 焦作市), (中通快运, 玉溪市), (中通快运, 甘孜藏族自治州), ...]

[1551 rows x 0 columns]

In [13]:
brand_ldr_type.reset_index(inplace =True)


,品牌,出发城市
0,COSCO-AIR,北京市
1,COSCO-AIR,西安市
2,中通快运,三明市
3,中通快运,三门峡市
4,中通快运,上海市
5,中通快运,上饶市
6,中通快运,中卫市
7,中通快运,临沧市
8,中通快运,丹东市
9,中通快运,丽水市


In [31]:
temp = brand_ldr_type.groupby("品牌").count()
temp.reset_index(inplace =True)
temp.columns =["name","value"]

In [41]:
temp.set_index("name")

,value
name,
COSCO-AIR,2
中通快运,151
中铁物流,118
传邦物流,40
佳吉快运,141
佳怡物流,37
兴铁物流,13
卡行天下,28
天地华宇,191


In [44]:
temp.to_json("./resultData/brand_ldr_city_count.json",orient='records')

In [97]:
brand_ldr.dtypes

品牌          object
出发城市        object
出发区域        object
到达城市        object
到达区域        object
时效         float64
线路类型        object
重货价格       float64
重货折扣       float64
轻货价格        object
轻货折扣       float64
最第一票         int64
最近一月成交量    float64
评价数量       float64
dtype: object

In [27]:
stripstr = lambda x: x.replace(",","") if isinstance(x, str) else x
temp2 = brand_ldr.applymap(stripstr)
temp2 = temp2.drop(['到达城市','出发城市','出发区域','到达区域','线路类型','最第一票','最近一月成交量','评价数量'],axis=1)
temp2["轻货价格"] = temp2["轻货价格"].astype("float64")

In [28]:
temp2 = temp2.groupby("品牌").mean().reset_index()
temp2_data ={
    "品牌":[],
    "时效":[],
    "重货价格":[],
    "重货折扣":[],
    "轻货价格":[],
    "轻货折扣":[]
}
for idx,row in temp2.iterrows():
    temp2_data["品牌"].append(row["品牌"]) 
    temp2_data["时效"].append(row["时效"]) 
    temp2_data["重货价格"].append(row["重货价格"]) 
    temp2_data["重货折扣"].append(row["重货折扣"]) 
    temp2_data["轻货价格"].append(row["轻货价格"]) 
    temp2_data["轻货折扣"].append(row["轻货折扣"]) 
with open("./mean.json",'wb') as fp:
    fp.write(json.dumps(temp2_data).encode("utf-8"))

In [32]:
temp2 = temp2.groupby("品牌").mean().reset_index()
temp2
brand_ldr

,品牌,出发城市,出发区域,到达城市,到达区域,时效,线路类型,重货价格,重货折扣,轻货价格,轻货折扣,最第一票,最近一月成交量,评价数量
0,天地华宇,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,NaN,108.0,经济快运,0.55,8.5,122,8.5,10,NaN,NaN
1,德邦,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,NaN,180.0,精准汽运（短）,0.59,8.6,123,8.5,20,NaN,NaN
2,远成快运,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,NaN,24.0,300公斤到1吨,0.60,9.0,120,9.0,201,NaN,NaN
3,天地华宇,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,NaN,72.0,定日达,0.72,8.5,159,8.5,15,NaN,NaN
4,德邦,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,NaN,154.0,精准卡航,0.77,8.6,161,8.5,40,NaN,NaN
5,德邦,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,陇川县,NaN,精准汽运（长）,0.88,8.5,185,8.5,25,NaN,NaN
6,德邦,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,盈江县,NaN,精准汽运（长）,0.97,8.5,203,8.5,25,NaN,NaN
7,远成快运,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,NaN,24.0,31-300公斤,0.98,9.0,196,9.0,47,NaN,NaN
8,德邦,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,陇川县,NaN,精准卡航,1.05,8.5,221,8.5,45,NaN,NaN
9,德邦,德宏傣族景颇族自治州,NaN,德宏傣族景颇族自治州,盈江县,NaN,精准卡航,1.15,8.5,241,8.5,45,NaN,NaN
